In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [3]:
seed = 428

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [4]:
def get_masked_input_and_labels(encoded_texts, n_cat):
    # For each sentence, mask each word one-by-one

    encoded_texts_masked = []
    y_labels = []

    for encoded_text in encoded_texts:
        for i in range(len(encoded_text)):
            encoded_text_masked = np.copy(encoded_text)
            y_label = encoded_text_masked[i]
            encoded_texts_masked.append(np.delete(encoded_text_masked, i))
            y_labels.append(np.array([y_label]))

    return np.array(encoded_texts_masked), np.array(y_labels)

In [5]:
#### K = number of countries = number of capitals = number of universities = number of mascots
#### S = number of words used by both topics
#### L = sentence length
#### q1, q2 = probability of having 1 or 2 pairs
#### embed_dim = dimension of embeddings
#### n_sentences = number of training sentences

def train_model(K, S, L, q1, q2, embed_dim, n_sentences):
    
    countries = ['country_' + str(i) for i in range(K)]
    capitals = ['capital_' + str(i) for i in range(K)]
    universities = ['university_' + str(i) for i in range(K)]
    mascots = ['mascot_' + str(i) for i in range(K)]
    randoms = ['random_' + str(i) for i in range(S)]

    vocabs = countries + capitals + universities + mascots + randoms
    vocab_map = {}

    for i in range(len(vocabs)):
        vocab_map[vocabs[i]] = i
        
    sentences = []
    sentences_number = []
    
    q0 = 1 - q1 - q2

    for i in range(n_sentences):

        sentence = []
        
        temp = npr.uniform()
        temp2 = npr.uniform()
        
        if temp2 <= q0:
            n_pairs = 0
        elif temp2 <= q0 + q1:
            n_pairs = 1
        else:
            n_pairs = 2
        
        if temp <= 0.5: ### country - capital
        
            pairs = np.random.choice(np.arange(K), n_pairs, replace = False)
            for pair in pairs:
                sentence.append(countries[pair])
                sentence.append(capitals[pair])        
                 
        else: ### university - mascot
            
            pairs = np.random.choice(np.arange(K), n_pairs, replace = False)
            for pair in pairs:
                sentence.append(universities[pair])
                sentence.append(mascots[pair])        
            
        sentence += list(np.random.choice(randoms, L - 2 * n_pairs, replace = False))  

        sentence_number = [vocab_map[i] for i in sentence]
        sentences.append(sentence)
        sentences_number.append(sentence_number)
        
    x_train = np.array(sentences_number)
    n_cat = len(vocab_map)
    x_masked_train, y_masked_labels_train = get_masked_input_and_labels(x_train, n_cat)
    
    callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
    inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
    word_embeddings = layers.Embedding(n_cat, embed_dim, name="word_embedding")(inputs)
    encoder_output = layers.GlobalAveragePooling1D()(word_embeddings)
    mlm_output = layers.Dense(n_cat, name="mlm_cls", activation="softmax", use_bias=False)(encoder_output)
    mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
    adam = Adam()
    mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)

    history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=500, batch_size=128, verbose=0)
    
    return sentences, vocab_map, mlm_model

In [6]:
def get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples):
    
    sentences, vocab_map, current_model = train_model(K, S, L, q1, q2, embed_dim, n_sentences)

    acc_capitals = []
    prob_capitals = []

    for _ in range(n_samples):
        sentence = []
        random_capitals = np.random.choice(np.arange(K), int(L/2), replace = False)
        for random_capital in random_capitals:
            sentence.append('country_' + str(random_capital))
            sentence.append('capital_' + str(random_capital))
        sentence = sentence[:-1]
        sentence_number = [vocab_map[i] for i in sentence]
        temp = keras.backend.function(inputs = current_model.layers[0].input, outputs = current_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        actual = vocab_map['capital_' + str(random_capitals[-1])]
        acc_capitals.append(1 if np.argsort(-1 * temp)[0][0] == actual else 0)
        prob_capitals.append(temp[0][vocab_map['capital_' + str(random_capitals[-1])]])
        
    acc_mascots = []
    prob_mascots = []

    for _ in range(n_samples):
        sentence = []
        random_mascots = np.random.choice(np.arange(K), int(L/2), replace = False)
        for random_mascot in random_mascots:
            sentence.append('university_' + str(random_mascot))
            sentence.append('mascot_' + str(random_mascot))
        sentence = sentence[:-1]
        sentence_number = [vocab_map[i] for i in sentence]
        temp = keras.backend.function(inputs = current_model.layers[0].input, outputs = current_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        actual = vocab_map['mascot_' + str(random_mascots[-1])]
        acc_mascots.append(1 if np.argsort(-1 * temp)[0][0] == actual else 0)
        prob_mascots.append(temp[0][vocab_map['mascot_' + str(random_mascots[-1])]])
        

    return sentences, current_model, vocab_map, (np.mean(acc_capitals), np.mean(prob_capitals)), \
                (np.mean(acc_mascots), np.mean(prob_mascots))

In [10]:
K = 10 # number of countries
L = 8 # sentence length
S = 20 # number of words used by both topics
embed_dim = 10 # CBOW embedding dimension
n_sentences = 50000 # number of sentences in the training set
n_samples = 1000

In [11]:
q0 = 0 # probability of having 0 pairs
q1 = 1 # probability of having 1 pair
q2 = 0 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(0.0, 0.0)
(0.0, 2.04923e-40)
(0.0, 0.0)
(0.0, 2.1100454e-37)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 4.123485e-37)
(0.0, 2.1834122e-37)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 4.1234850755596704e-38)
(0.0, 4.295506877559935e-38)


In [12]:
q0 = 0 # probability of having 0 pairs
q1 = 0 # probability of having 1 pair
q2 = 1 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(0.0, 4.3124267e-18)
(0.0, 4.562811e-18)
(0.0, 5.1693726e-17)
(0.0, 1.3995944e-17)
(0.0, 1.3545231e-21)
(0.0, 4.9939878e-23)
(0.0, 7.159587e-15)
(0.0, 1.9240127e-15)
(0.0, 1.0670042e-28)
(0.0, 5.1297958e-26)
(0.0, 2.0058213e-19)
(0.0, 2.380234e-23)
(0.0, 4.492576e-19)
(0.0, 3.749769e-20)
(0.0, 1.1131691e-11)
(0.0, 3.2166967e-12)
(0.0, 3.8496327e-17)
(0.0, 2.6693755e-17)
(0.0, 1.1717427e-13)
(0.0, 9.53807e-15)
(0.0, 1.1256120064239628e-12)
(0.0, 3.228204030471607e-13)


In [13]:
q0 = 1/2 # probability of having 0 pairs
q1 = 1/2 # probability of having 1 pair
q2 = 0 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 0.9999986)
(1.0, 0.9999994)
(1.0, 0.9999984)
(1.0, 0.9999959)
(1.0, 1.0)
(1.0, 0.99999964)
(1.0, 0.9999999)
(1.0, 0.9999999)
(1.0, 0.9999995)
(1.0, 0.9999995)
(1.0, 0.99999917)
(1.0, 0.9999999)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 0.9999987)
(1.0, 0.9999996)
(1.0, 0.9999999)
(1.0, 0.99999994)
(1.0, 1.0)
(1.0, 0.99999934)
(0.9999999999999999, 0.9999994158744813)
(0.9999999999999999, 0.9999993085861205)


In [14]:
q0 = 1/2 # probability of having 0 pairs
q1 = 0 # probability of having 1 pair
q2 = 1/2 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 0.9999999)
(1.0, 0.9999989)
(1.0, 1.0)
(1.0, 0.99999964)
(1.0, 0.99999964)
(1.0, 1.0)
(1.0, 0.9999985)
(1.0, 0.99999976)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 0.9999994)
(1.0, 0.9999999)
(1.0, 0.9999999)
(1.0, 1.0)
(1.0, 0.9999995)
(1.0, 0.9999984)
(1.0, 1.0)
(1.0, 0.9999999)
(1.0, 0.99999964)
(1.0, 0.99998814)
(0.9999999999999999, 0.9999996483325958)
(0.9999999999999999, 0.9999984622001649)


In [17]:
q0 = 0 # probability of having 0 pairs
q1 = 1/2 # probability of having 1 pair
q2 = 1/2 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 0.9998244)
(1.0, 0.99998164)
(1.0, 0.99999726)
(1.0, 0.999532)
(1.0, 0.9998862)
(1.0, 1.0)
(1.0, 0.99999887)
(1.0, 1.0)
(1.0, 0.99999535)
(1.0, 0.99999535)
(1.0, 0.9995443)
(1.0, 0.99999964)
(1.0, 0.9997944)
(1.0, 0.9999907)
(1.0, 0.99614495)
(1.0, 0.99980813)
(1.0, 0.99999374)
(1.0, 0.9985459)
(1.0, 0.9999816)
(1.0, 0.9996882)
(0.9999999999999999, 0.9995161116123199)
(0.9999999999999999, 0.9997541546821593)


In [18]:
q0 = 1/3 # probability of having 0 pairs
q1 = 1/3 # probability of having 1 pair
q2 = 1/3 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 0.9999955)
(1.0, 0.9999993)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 0.9999999)
(1.0, 0.99999976)
(1.0, 1.0)
(1.0, 0.9999999)
(1.0, 0.99999976)
(1.0, 1.0)
(1.0, 0.9999989)
(1.0, 1.0)
(1.0, 0.9993109)
(1.0, 1.0)
(1.0, 0.99999976)
(1.0, 0.9999999)
(1.0, 0.99999976)
(1.0, 0.99999976)
(0.9999999999999999, 0.999930453300476)
(0.9999999999999999, 0.9999998569488524)
